# NLP Package
## Topic : Fake Reports Identifier
### Team
* 18pt15 - Jeyakumaran M
* 18pt41 - Charan K R

In [1]:
# Libraries to be imported
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from matplotlib import pyplot as plt
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.feature_extraction.text import HashingVectorizer
import itertools
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn

In [2]:
#Getting Data
df1 = pd.read_fwf('F:/DataScience/NLP/False-Report-Detection/data/data_doctor_truthful.txt', header=None)
df2 = pd.read_fwf('F:/DataScience/NLP/False-Report-Detection/data/data_doctor_deceptive.txt', header=None)
df1[1]=1
df2[1]=0
frames = [df1, df2]
df = pd.concat(frames)
df.drop(df.iloc[:, 2:94], inplace = True, axis = 1)
df.columns = ['text','label']

In [3]:
#Process Data
df['text'] = [entry.lower() for entry in df['text']]
df['text']= [word_tokenize(entry) for entry in df['text']]
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
df.reset_index(inplace = True, drop = True) 
for index,entry in enumerate(df['text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    df.loc[index,'text_final'] = str(Final_words)
X_train, X_test, y_train, y_test = train_test_split(df['text_final'], df['label'], test_size=0.33)
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7, max_features=5000) # This removes words which appear in more than 70% of the articles
# Fit and transform the training data 
tfidf_vectorizer.fit(df['text_final'])
tfidf_train = tfidf_vectorizer.transform(X_train)
# Transform the test set # Initialize the `tfidf_vectorizer`
tfidf_test = tfidf_vectorizer.transform(X_test)
count_vectorizer = CountVectorizer(stop_words='english')  # Initialize the `count_vectorizer`
# Fit and transform the training data 
count_train = count_vectorizer.fit_transform(X_train) # Learn the vocabulary dictionary and return term-document matrix.
# Transform the test set 
count_test = count_vectorizer.transform(X_test)
print(tfidf_vectorizer.vocabulary_)
print(tfidf_train)
count_df = pd.DataFrame(count_train.A, columns=count_vectorizer.get_feature_names())
tfidf_df = pd.DataFrame(tfidf_train.A, columns=tfidf_vectorizer.get_feature_names())
difference = set(count_df.columns) - set(tfidf_df.columns)

{'aboolian': 10, 'staff': 2379, 'initial': 1293, 'expectation': 916, 'maintain': 1523, 'high': 1196, 'level': 1444, 'professionalism': 1955, 'make': 1525, 'feel': 971, 'comfortable': 484, 'fact': 947, 'ease': 799, 'definitely': 671, 'important': 1267, 'element': 825, 'surgical': 2466, 'procedure': 1947, 'lose': 1498, 'pound': 1894, 'need': 1654, 'breast': 338, 'lift': 1453, 'enlargement': 853, 'consult': 558, 'doctor': 761, 'meet': 1565, 'evident': 887, 'right': 2152, 'choice': 442, 'trust': 2610, 'life': 1448, 'want': 2748, 'result': 2135, 'different': 720, 'recommend': 2044, 'interested': 1324, 'cosmetic': 591, 'surgery': 2465, 'california': 385, 'land': 1417, 'calgary': 383, 'immigrant': 1262, 'fortunate': 1033, 'greyvenstein': 1117, 'nice': 1673, 'care': 400, 'pregnancy': 1911, 'baby': 250, 'birth': 301, 'nervous': 1665, 'time': 2570, 'mom': 1616, 'meeting': 1566, 'determined': 704, 'ca': 380, 'thank': 2532, 'subordinate': 2436, 'rockyview': 2160, 'hospital': 1227, 'ofcourse': 1715

In [4]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
clf = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
clf.fit(tfidf_train, y_train)       # Fit SVM classifier according to X, y
pred = clf.predict(tfidf_test)      # Perform classification on an array of test vectors X.
score = metrics.accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)
precision = metrics.precision_score(y_test, pred)
print("precision:   %0.3f" % precision)
cm = metrics.confusion_matrix(y_test, pred)
sensitivity = cm[0,0]/(cm[0,0]+cm[0,1])
print('Sensitivity : ', sensitivity )
specificity = cm[1,1]/(cm[1,0]+cm[1,1])
print('Specificity : ', specificity )
f1_score = metrics.f1_score(y_test, pred)
print("F1_Score:   %0.3f" % f1_score)
print("The confusion matrix is  ",cm)

accuracy:   0.918
precision:   0.931
Sensitivity :  0.9663865546218487
Specificity :  0.8307692307692308
F1_Score:   0.878
The confusion matrix is   [[115   4]
 [ 11  54]]


In [5]:
#RandomForest
clf=RandomForestClassifier(n_estimators=500)
clf.fit(tfidf_train, y_train)       # Fit  Random Forest classifier according to X, y
pred = clf.predict(tfidf_test)      # Perform classification on an array of test vectors X.
score = metrics.accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)
precision = metrics.precision_score(y_test, pred)
print("precision:   %0.3f" % precision)
cm = metrics.confusion_matrix(y_test, pred)
sensitivity = cm[0,0]/(cm[0,0]+cm[0,1])
print('Sensitivity : ', sensitivity )
specificity = cm[1,1]/(cm[1,0]+cm[1,1])
print('Specificity : ', specificity )
f1_score = metrics.f1_score(y_test, pred)
print("F1_Score:   %0.3f" % f1_score)
print("The confusion matrix is  ",cm)

accuracy:   0.755
precision:   0.812
Sensitivity :  0.9495798319327731
Specificity :  0.4
F1_Score:   0.536
The confusion matrix is   [[113   6]
 [ 39  26]]


In [6]:
#Naive Bayes with Tf and idf
clf = MultinomialNB() 
clf.fit(tfidf_train, y_train)       # Fit Naive Bayes classifier according to X, y
pred = clf.predict(tfidf_test)      # Perform classification on an array of test vectors X.
score = metrics.accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)
precision = metrics.precision_score(y_test, pred)
print("precision:   %0.3f" % precision)
cm = metrics.confusion_matrix(y_test, pred)
sensitivity = cm[0,0]/(cm[0,0]+cm[0,1])
print('Sensitivity : ', sensitivity )
specificity = cm[1,1]/(cm[1,0]+cm[1,1])
print('Specificity : ', specificity )
f1_score = metrics.f1_score(y_test, pred)
print("F1_Score:   %0.3f" % f1_score)
print("The confusion matrix is  ",cm)

accuracy:   0.679
precision:   1.000
Sensitivity :  1.0
Specificity :  0.09230769230769231
F1_Score:   0.169
The confusion matrix is   [[119   0]
 [ 59   6]]


In [7]:
#Naive Bayes with count vectorizer
clf.fit(count_train, y_train)
pred = clf.predict(count_test)
score = metrics.accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)
precision = metrics.precision_score(y_test, pred)
print("precision:   %0.3f" % precision)
cm = metrics.confusion_matrix(y_test, pred)
sensitivity = cm[0,0]/(cm[0,0]+cm[0,1])
print('Sensitivity : ', sensitivity )
specificity = cm[1,1]/(cm[1,0]+cm[1,1])
print('Specificity : ', specificity )
f1_score = metrics.f1_score(y_test, pred)
print("F1_Score:   %0.3f" % f1_score)
print("The confusion matrix is  ",cm)

accuracy:   0.913
precision:   0.980
Sensitivity :  0.9915966386554622
Specificity :  0.7692307692307693
F1_Score:   0.862
The confusion matrix is   [[118   1]
 [ 15  50]]


In [8]:
#Ridge Classifier
clf=RidgeClassifier()
clf.fit(tfidf_train, y_train)       # Fit Ridge Regression classifier according to X, y
pred = clf.predict(tfidf_test)      # Perform classification on an array of test vectors X.
score = metrics.accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)
precision = metrics.precision_score(y_test, pred)
print("precision:   %0.3f" % precision)
cm = metrics.confusion_matrix(y_test, pred)
sensitivity = cm[0,0]/(cm[0,0]+cm[0,1])
print('Sensitivity : ', sensitivity )
specificity = cm[1,1]/(cm[1,0]+cm[1,1])
print('Specificity : ', specificity )
f1_score = metrics.f1_score(y_test, pred)
print("F1_Score:   %0.3f" % f1_score)
print("The confusion matrix is  ",cm)

accuracy:   0.891
precision:   0.941
Sensitivity :  0.9747899159663865
Specificity :  0.7384615384615385
F1_Score:   0.828
The confusion matrix is   [[116   3]
 [ 17  48]]


# The End